# <font color=black> Figure 1 Spinal cord morphometry </font>
<hr style="border:1px solid black">

### Imports

In [ ]:
import glob, os, sys, json
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project"
sys.path.append(main_dir + "/code/")
from sc_structural_analyses import StructuralMetrics
from sim_matrix import Matrix

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load config file ------------------------------------------------------------
config_file=main_dir + '/config/analyses/brsc_structural.json'
with open(config_file) as config_f: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_f) # load config file should be open first and the path inside modified

# plotting
from plotting import Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
plot=Plotting(config_file,"test")

#statistics
from scipy import stats
from brsc_statistics import Statistics 
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
stat_func=Statistics(config=config,ana_dir="",analysis="")


# Atlas_labels ------------------------------
atlas_labels_f=glob.glob(config["project_dir"] + config['template']["spinalcord"]['atlas'].split("order")[0] + "labels.txt")[0]
atlas_labels=np.genfromtxt(atlas_labels_f, usecols=0,skip_header=1, dtype="S", delimiter="\t", encoding=None)
atlas_labels = np.array([label.decode("utf-8") for label in atlas_labels])


<hr style="border:1px solid black">

## <font color=#0B7CC3> A. Extract features

In [ ]:
sub_metrics={"MTR":["MTR"],"T2s":["T2s_vx_gm","T2s_vx_wm"],"DWI":["FA","RD","AD","MD"]}
#sub_metrics={"T2s":["T2s_vx_gm","T2s_vx_wm"]}

df_metrics={}
atlas_f=[]

for contrast in sub_metrics:
    df_metrics[contrast]={}
    mask_path={}
    for sub_metric in sub_metrics[contrast]:
        print(sub_metric)
        files_indivspace=[]
        if len(sub_metrics[contrast])>1:
            metrics=StructuralMetrics(config,contrast=contrast,metric=sub_metric) # initialize the function
        else:
            metrics=StructuralMetrics(config,contrast=contrast)
        atlas_f=[]
        for ID_nb, ID in enumerate(config["participants_IDs_" + contrast]):
            preproc_dir= config["preprocess_dir"]["bmpd"] if ID[0]=="P" else config["preprocess_dir"]["stratals"]
            if contrast=="T2s":
                if sub_metric=="T2s_gm" or sub_metric=="T2s_wm":
                    atlas_f.append(preproc_dir + config["indiv_space"][sub_metric + "_atlas"].format(ID))
                    files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][contrast].format(ID))[0])
                
                else:
                    atlas_f.append(preproc_dir + config["indiv_space"][contrast+ "_atlas"].format(ID))
                    files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][sub_metric].format(ID))[0])
                
            elif contrast=="MTR":
                files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][contrast].format(ID))[0])
                
                if sub_metric=="MTR_gm" or sub_metric=="MTR_wm":
                    atlas_f.append(preproc_dir + config["indiv_space"][sub_metric + "_atlas"].format(ID))

                else:
                    atlas_f.append(preproc_dir + config["indiv_space"][contrast + "_atlas"].format(ID))

            elif contrast=="DWI":
                files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][sub_metric].format(ID))[0])
                atlas_f.append(preproc_dir + config["indiv_space"][contrast + "_atlas"].format(ID))

        measure="mean"
        if sub_metric=="T2s_vx_gm" or sub_metric=="T2s_vx_wm":
            measure="count"


        df_metrics[contrast][sub_metric]=metrics.extract_metric_rois(IDs=config["participants_IDs_" + contrast],
                            input_f=files_indivspace,
                            atlas_f=atlas_f,
                            atlas_labels=atlas_labels,
                            metric=sub_metric,
                            measure=measure,
                            space="indiv_space",
                            tag=""  ,norm=False,redo=False,verbose=1)

    
        df_metrics[contrast][sub_metric]=df_metrics[contrast][sub_metric][df_metrics[contrast][sub_metric]["levels"]!="C1"].reset_index(drop=True)
        df_metrics[contrast][sub_metric]["rois_couple"]=df_metrics[contrast][sub_metric]["ventro_dorsal"] + "_" + df_metrics[contrast][sub_metric]["right_left"]
    if contrast == "T2s" :
        if all(x in sub_metrics["T2s"] for x in ["T2s_gm", "T2s_wm"]):
            gm = df_metrics["T2s"]["T2s_gm"].get("T2s_gm")
            wm = df_metrics["T2s"]["T2s_wm"].get("T2s_wm")
            df_metrics["T2s"]["T2s_gm"]["wm_gm_r"] = wm / gm

    if contrast == "MTR" :
        if all(x in sub_metrics[contrast] for x in ["MTR_gm", "MTR_wm"]):
            gm = df_metrics[contrast]["MTR_gm"].get("MTR_gm")
            wm = df_metrics[contrast]["MTR_wm"].get("MTR_wm")
            df_metrics[contrast]["MTR_gm"]["wm_gm_r"] =  wm / gm


## Create group level nifti image

In [ ]:
sub_metrics={"MTR":["MTR"],"T2s":["T2s"],"DWI":["FA","RD","AD","MD"]}
import brsc_utils

for contrast in sub_metrics:
    output_dir=config["project_dir"] + config["analysis_dir"][contrast] + "2_second_level/nifti/"
    os.makedirs(output_dir,exist_ok=True)
    for sub_metric in sub_metrics[contrast]:
        
        for group in ["ALL","YA","MA","OA"]:
            i_files_dir=[]
            commun_ids=sorted(set(config["participants_IDs_" + contrast]) & set(config["participants_IDs_" + group]))
            for ID_nb, ID in enumerate(commun_ids):
                preproc_dir= config["preprocess_dir"]["bmpd"] if ID[0]=="P" else config["preprocess_dir"]["stratals"]
                split=config["PAM50_space"][sub_metric].format(ID).split("/")[0:-1][-1]
                i_files_dir.append(preproc_dir+ config["PAM50_space"][sub_metric].format(ID).split(split)[0]+f"/{split}/")
                
            brsc_utils.group_mean_img(IDs=commun_ids,
                              i_dir=i_files_dir,
                              o_dir=output_dir,
                              prefix_tag="sub-",
                              suffix_tag=config["PAM50_space"][sub_metric].format(ID).split("/")[-1].split(".nii.gz")[0],
                              remove_4d=True,
                             tag=sub_metric +"_" + group,
                             redo=False)

<hr style="border:1px solid black">

## <font color=#0B7CC3> C. Compute similarity matrix

In [ ]:
# Step 1: Find common IDs
common_ids = sorted(set(config["participants_IDs_T2s"]) & set(config["participants_IDs_MTR"]) & set(config["participants_IDs_DWI"]))

# Step 2: Filter each DataFrame to only keep rows with those common IDs
df_all=df_metrics["MTR"]["MTR"][df_metrics["MTR"]["MTR"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_gm=df_metrics["T2s"]["T2s_vx_gm"][df_metrics["T2s"]["T2s_vx_gm"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_wm=df_metrics["T2s"]["T2s_vx_wm"][df_metrics["T2s"]["T2s_vx_wm"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_fa=df_metrics["DWI"]["FA"][df_metrics["DWI"]["FA"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_rd=df_metrics["DWI"]["RD"][df_metrics["DWI"]["RD"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_md=df_metrics["DWI"]["MD"][df_metrics["DWI"]["MD"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_ad=df_metrics["DWI"]["AD"][df_metrics["DWI"]["AD"]["IDs"].isin(common_ids)].reset_index(drop=True)
df_all["gm"]=df_gm["T2s_vx_gm"];df_all["wm"]=df_wm["T2s_vx_wm"];#df_all["gm_wm_r"]=df_gmwm["gm_wm_r"]
df_all["FA"]=df_fa["FA"];df_all["RD"]=df_rd["RD"]
df_all["MD"]=df_md["MD"];df_all["AD"]=df_ad["AD"]
df_all_ventral=df_all[df_all['ventro_dorsal']=="ventral"]
df_all_dorsal=df_all[df_all['ventro_dorsal']=="dorsal"]

In [ ]:
atlas_labelsC2C7=atlas_labels[4:] # remove C1 level 
common_ids = sorted(set(config["participants_IDs_T2s"]) & set(config["participants_IDs_MTR"]) & set(config["participants_IDs_DWI"]))
output_dir=config["project_dir"] +  "/figures/f01_structural/revision_R1/spinalcord/similarity/"
ana_matrix=Matrix(config=config,IDs=common_ids,output_dir=output_dir)
col_selected = ["MTR","AD","MD","RD","FA","gm","wm"]


all_sim_matrix, mean_sim_matrix,df_sim=ana_matrix.compute_similarity_matrix(data_df=df_all,column_labels=col_selected ,atlas_labels=atlas_labelsC2C7,
                                                                            scaling_method="robust_sigmoid",redo=False)

colors = ["#F2ECDF","#C3E1DD","#93D6DB","#62B7D9","#3197D6","#0B7CC3","#0168A7"]  # blue → white → red
custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)
output_dir=config["project_dir"] +  "/figures/f01_structural/preprint2025/figures/"
plot.plot_heatmap(matrix=mean_sim_matrix,
                  vmax=1,vmin=-0.3,
                  cmap=custom_cmap,
            xticklabels=atlas_labelsC2C7,yticklabels=atlas_labelsC2C7,
                    labels=atlas_labels,
                  output_f=config["project_dir"] +  "/figures/f01_structural/revision_R1/sim_matrix_morpho.pdf",
                  save=False)
plt.show()
#mean_df.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_1c_intra_vs_inter.csv", index=False)

In [ ]:

mean_df=df_sim.groupby(["IDs","age","sex","betwith_labels"])[["sim"]].mean().reset_index()
# ---- compute within versus between t-test
t_test=stats.ttest_rel(mean_df[mean_df["betwith_labels"]=="intra"]["sim"],mean_df[mean_df["betwith_labels"]=="inter"]["sim"])
print("t(" + str(t_test.df) + "): " + str(np.round(t_test.statistic,2)) + " p-value: " + str(t_test.pvalue))

# ---- plot within and between similarity
plot.boxplots(df=mean_df,
              x_data="betwith_labels",x_order=["intra","inter"],
                  indiv_values=True,#invers_axes=True,
                  palette=["#0168A7","#93D6DB"],#output_dir=config['main_dir'] + config['analysis_dir']['spinalcord'] + '/figures/',
                  #output_tag='corr_' + group_name,
              height=5,aspect=0.3,
                  y_data="sim",
              ymin=-0.2, ymax=1.2,
              output_dir=config["project_dir"] +  "/figures/f01_structural/revision_R1/",
              output_tag="sim_intra-inter",
              save=False)
plt.show()
#mean_df.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_1c_intra_vs_inter.csv", index=False)

<hr style="border:1px solid black">

## <font color=#0B7CC3> D. Age and sex effects


In [ ]:
output_dir_table=config["project_dir"] +  "/figures/f01_structural/revision_R1/spinalcord/"

sub_metrics={"MTR":["MTR"],"T2s":["T2s_vx_gm","T2s_vx_wm"],"DWI":["FA","RD","AD","MD"]}
df_mean_metrics={};
mean_results = [];roi_results=[];
df_reg_all=pd.DataFrame([{"Metric": "","ROI": "","Predictor": "","β": "","95% Bootstrap CI": "","t-value": "","p-value": "","p-fdr":""}])
dfs = []
for contrast in sub_metrics:
    df_mean_metrics[contrast]={}
    for sub_metric in sub_metrics[contrast]:
        if sub_metric == "T2s_wm":
            continue  # skip this sub_metric and move to the next one
        
        df_mean_metrics[sub_metric]={}
        if sub_metric in ["T2s_gm", "MTR_gm"]:
            y = "wm_gm_r"
        else:
            y = sub_metric
        
        signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,ci_r2_age,  ci_beta_age, ci_beta_cov =stat_func.regression_model(df=df_metrics[contrast][sub_metric].dropna(),y=y,predictor="age",covariates=["sex"],random='IDs')
        mean_results.append({"contrast": contrast,"sub_metric": y,"signed_r2": signed_r2,"p_age": p_age,"p_sex": p_sex,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
        
        for roi in np.unique(df_metrics[contrast][sub_metric]["rois_couple"]):
            if not os.path.exists(output_dir_table + "stats_sc_"+sub_metric+"_"+roi+"_by_horn.csv"):
                roi_subresults=[]
                df_roi_metrics=df_metrics[contrast][sub_metric][df_metrics[contrast][sub_metric]["rois_couple"]==roi]
                signed_r2_2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,ci_r2_age, ci_beta_age, ci_beta_sex =stat_func.regression_model(df=df_roi_metrics.dropna(),y=y,predictor="age",covariates=["sex"],random='IDs', n_bootstrap=1000)
                roi_subresults.append({"Metric": sub_metric,"ROI": roi,"Predictor": "age","β": beta_age,"95% Bootstrap CI": ci_beta_age,"t-value": stat_age,"p-value": p_age,"p-fdr": p_age,})
                roi_subresults.append({"Metric": sub_metric,"ROI": roi,"Predictor": "sex","β": beta_sex,"95% Bootstrap CI": ci_beta_sex,"t-value": stat_sex,"p-value": p_sex,"p-fdr": p_sex,})
                roi_subresults_df= pd.DataFrame(roi_subresults)

                roi_subresults_df.to_csv(output_dir_table + "stats_sc_"+sub_metric+"_"+roi+"_by_horn.csv", index=False)
            
            else:
                roi_subresults_df = pd.read_csv(output_dir_table + "stats_sc_"+sub_metric+"_"+roi+"_by_horn.csv")

            dfs.append(roi_subresults_df)
            df_reg_all = pd.concat(dfs, ignore_index=True)#pd.concat([df_all, roi_subresults_df], ignore_index=True)


# FDR correction
df_reg_all["p-fdr"] = None  # initialize column

if not os.path.exists(output_dir_table + "stats_sc_all_by_horn.csv"):
    for predictor in df_reg_all["Predictor"].unique():
        mask = df_reg_all["Predictor"] == predictor
        pvals = df_reg_all.loc[mask, "p-value"].values
    
        _, pvals_fdr, _, _ = multipletests(pvals,alpha=0.05,method="fdr_bh")
    
        df_reg_all.loc[mask, "p-fdr"] = pvals_fdr
    df_reg_all.to_csv(output_dir_table + "stats_sc_all_by_horn.csv", index=False)
else:
    df_reg_all = pd.read_csv(output_dir_table + "stats_sc_all_by_horn.csv")
#df_all
for roi in np.unique(df_reg_all["ROI"]):
    print(roi + " absolute mean t-value: " + str(np.round(np.mean(np.abs(df_reg_all[(df_reg_all["ROI"]==roi)
                  &(df_reg_all["Predictor"]=='age')]["t-value"])),3)) + " ± " +
         str(np.round(np.std(np.abs(df_reg_all[(df_reg_all["ROI"]==roi)
                  &(df_reg_all["Predictor"]=='age')]["t-value"])),2)))

## <font color=#0B7CC3> D. Age and sex effects

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

output_file = output_dir_table + "All_metrics_vs_age_LMM_single_page.pdf"

# Count total rows and columns for the entire figure
total_rows = sum(len(sub_metrics[contrast]) for contrast in sub_metrics)
all_rois = []
for contrast in sub_metrics:
    for sub_metric in sub_metrics[contrast]:
        rois = np.unique(df_metrics[contrast][sub_metric]["rois_couple"])
        all_rois.extend(rois)
all_rois = np.unique(all_rois)
total_cols = len(all_rois)

fig, axes = plt.subplots(total_rows, total_cols, figsize=(total_cols*3, total_rows*3), squeeze=False)

row_offset = 0
for contrast in sub_metrics:
    metrics = sub_metrics[contrast]
    for i, sub_metric in enumerate(metrics):
        for j, roi in enumerate(all_rois):
            ax = axes[row_offset + i, j]

            df_roi = df_metrics[contrast][sub_metric][
                df_metrics[contrast][sub_metric]["rois_couple"] == roi
            ].copy()
            df_roi = df_roi.dropna(subset=[sub_metric, 'age', 'sex'])
            df_roi['sex_bin'] = df_roi['sex'].map({'F': 0, 'M': 1})
            df_roi['IDs'] = df_roi['IDs'].astype('category')

            md = smf.mixedlm(f"{sub_metric} ~ age + sex_bin", df_roi, groups=df_roi['IDs'])
            mdf = md.fit()

            df_subject = df_roi.groupby(['IDs', 'age', 'sex_bin'], observed=False).agg({sub_metric: 'mean'}).reset_index()
            ax.scatter(df_subject['age'], df_subject[sub_metric], color=color[contrast], alpha=0.7, s=30)
            p_age = mdf.pvalues.get('age', np.nan)
            if p_age < 0.05 :
                if p_age < 0.001:
                    stars = '***'
                elif p_age < 0.01:
                    stars = '**'
                elif p_age < 0.05:
                    stars = '*'

                # mark the earliest age with significant effect
                sig_age = df_roi['age'].min()
                sig_value = mdf.predict(pd.DataFrame({'age':[sig_age],'sex_bin':[df_roi['sex_bin'].mode()[0]]}))[0]
                ax.text(sig_age, sig_value, stars,  fontsize=12, ha='center', va='bottom')
                
            age_range = np.linspace(df_roi['age'].min(), df_roi['age'].max(), 100)
            pred_df = pd.DataFrame({
                'age': age_range,
                'sex_bin': df_roi['sex_bin'].mode()[0]
            })
            pred_values = mdf.predict(pred_df)
            ax.plot(age_range, pred_values, color=color[contrast], lw=2)

            if row_offset + i == total_rows - 1:
                ax.set_xlabel('Age')
            if j == 0:
                ax.set_ylabel(sub_metric)
            ax.set_title(str(roi), fontsize=8)
            ax.tick_params(axis='both', which='major', labelsize=7)

    row_offset += len(metrics)

plt.tight_layout()

# Save as a single-page PDF
plt.tight_layout(rect=[0,0.1,1,0.80])
plt.savefig(output_file, dpi=300, bbox_inches='tight')
plt.close(fig)


In [ ]:
pivot = df_reg_all[(df_reg_all["Predictor"]=="age")].pivot(index="ROI", columns="Metric", values="t-value")
feat_order=["MTR","T2s_vx_gm","T2s_vx_wm","FA","RD","AD","MD"]
colors = ["#2C1562","#61508A","#6F5F94","#968AB1","#FFFFFF","#FACF80","#F7B740","#DB9930","#B4730A"]
colors = ["#0168A7","#0B7CC3","#62B7D9","#C3E1DD","#FFFFFF","#E5E5E5","#A1A1A1","#525252","#262626"]
#colors = ["#262626","#525252","#A1A1A1","#E5E5E5","#FFFFFF","#C3E1DD","#62B7D9","#0B7CC3","#0168A7"]

#["#F2ECDF","#C3E1DD","#93D6DB","#62B7D9","#3197D6","#0B7CC3","#0168A7"
 # blue → white → red
#colors = ["#2C1562","#61508A","#968AB1","#FFFFFF","#FACF80","#F7B740","#F49E00"FCB44A",]  # blue → white → red

custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)

pivot = pivot.reindex(columns=feat_order)

plt.figure(figsize=(6, 3))
sns.heatmap(pivot, cmap=custom_cmap, center=0, vmin=-5, vmax=6, fmt=".2f")
plt.title("T-values per ROI and Metric")
plt.tight_layout()
#plt.savefig(config["project_dir"] +  '/figures/f01_structural/revision_R1/age_effect_tvalues_quad.pdf', format='pdf')
plt.show()

#pivot.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_1d_morpho_aging.csv", index=False)

In [ ]:
# compute similarity for significant features only
col_selected = ['MTR','wm', 'gm', 'FA', 'RD', 'AD']
all_sim_matrix_age, mean_sim_matrix_age,df_sim_age=ana_matrix.compute_similarity_matrix(data_df=df_all,column_labels=col_selected ,atlas_labels=atlas_labelsC2C7,
                                                                                        scaling_method="robust_sigmoid",
                                                                                        tag="_signif_age",redo=True)


###  <font color=#16A89A> B. Predictive models

In [ ]:
output_dir=config["project_dir"] +  '/figures/f01_structural/revision_R1/'

from brsc_classification import Classification
classif=Classification(config)
mean_df_VD=df_all.groupby(["IDs","ventro_dorsal"])[["age","MTR","AD","MD","RD","FA","gm","wm"]].mean().reset_index()
mean_df_RL=df_all.groupby(["IDs","right_left"])[["age","MTR","AD","MD","RD","FA","gm","wm"]].mean().reset_index()

model_param = {"alpha": 1.2, "max_iter": 1000, "l1_ratio": 1} 
metrics_sc={};prediction_sc={}; importances={};
for roi in ["dorsal","ventral"]:
    print(roi)
    df=mean_df_VD[mean_df_VD["ventro_dorsal"]==roi]
    metrics_sc[roi],prediction_sc[roi], importances[roi]=classif.prediction_model(df=df,
                                                                   target_col='age',
                                                                   parcels_col="ventro_dorsal",
                                                                   model_param=model_param,
                                                                   feature_cols=["MTR","AD","MD","RD","FA","gm","wm"],
                                                                   model_name="ElasticNet")
    plot.lmplots(df=prediction_sc[roi],color=["#000000"],
                 x_data="y_true",
                 y_data="y_pred",xmin=15,xmax=85,
                 ymin=15, ymax=85,xy_plot=True,
                  output_dir=output_dir,
                 output_tag="age_prediction_" + roi,
                 save=False)
    print(importances[roi])
                 
    plt.show()
    df = importances[roi]
    custom_order = ["FA", "RD", "AD","MD",  "gm", "wm", "MTR"]
    df = df.set_index("Feature").loc[custom_order].reset_index()
    plot.donutplots(y_data=df["PercCoefficient"],
                    labels=custom_order,
                    palette=["#B14263","#BE637E","#CB8499","#E4C5CF","#dd9c07","#DDB350","#16A89A"],
                   output_dir=output_dir,
                 output_tag="age_prediction_imp_donut_" + roi,
                 save=False)
    
for roi in ["right","left"]:
    df=mean_df_RL[mean_df_RL["right_left"]==roi]
    metrics_sc[roi],prediction_sc[roi], importances[roi]=classif.prediction_model(df=df,
                                                                   target_col='age',
                                                                   parcels_col="right_left",
                                                                   model_param=model_param,
                                                                   feature_cols=["MTR","AD","MD","RD","FA","gm","wm"],
                                                                   model_name="ElasticNet")


#mean_df_VD.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_1E_prediction_age.csv", index=False)

In [ ]:

results_file = output_dir + "spinal_age_prediction_perm_results.pkl"

# Check if file exists
if os.path.exists(results_file):
    print(f"Loading saved results from {results_file}")
    with open(results_file, "rb") as f:
        metrics_sc, prediction_sc, importances, perm_results = pickle.load(f)
else:
    from brsc_classification import Classification
    classif=Classification(config)
    
    mean_df_VD=df_all.groupby(["IDs","ventro_dorsal"])[["age","MTR","AD","MD","RD","FA","gm","wm"]].mean().reset_index()
    mean_df_RL=df_all.groupby(["IDs","right_left"])[["age","MTR","AD","MD","RD","FA","gm","wm"]].mean().reset_index()

    model_param = {"alpha": 1.2, "max_iter": 1000, "l1_ratio": 1} 
    metrics_sc, prediction_sc, importances, perm_results = {}, {}, {}, {}
    n_perm = 1000
        


    for roi in ["dorsal","ventral"]:
        print(f"Processing ROI: {roi}")
        df_roi=mean_df_VD[mean_df_VD["ventro_dorsal"]==roi]
        metrics_obs, pred_obs, imp_obs = classif.prediction_model(
            df=df_roi,
            target_col="age",
            parcels_col="ventro_dorsal",
            feature_cols=["MTR","AD","MD","RD","FA","gm","wm"],
            model_param=model_param,
            model_name="ElasticNet"
        )
    
        r2_obs  = metrics_obs["r2"]
        mae_obs = metrics_obs["mae"]
    
        # -------------------------------
        # Permutation test
        # -------------------------------
        r2_perm  = np.zeros(n_perm)
        mae_perm = np.zeros(n_perm)
    
        subj_age = (
            df_roi[["IDs", "age"]]
            .drop_duplicates()
            .set_index("IDs")
        )
    
        for i in range(n_perm):
            shuffled_age = np.random.permutation(subj_age["age"].values)
            age_map = dict(zip(subj_age.index, shuffled_age))
    
            df_perm = df_roi.copy()
            df_perm["age"] = df_perm["IDs"].map(age_map)
    
            metrics_p, _, _ = classif.prediction_model(
                df=df_perm,
                target_col="age",
                parcels_col="ventro_dorsal",
                feature_cols=["MTR","AD","MD","RD","FA","gm","wm"],
                model_param=model_param,
                model_name="ElasticNet", verbose=False,
            )
    
            r2_perm[i]  = metrics_p["r2"]
            mae_perm[i] = metrics_p["mae"]
    
        # -------------------------------
        # Permutation p-values
        # -------------------------------
        p_perm_r2  = (np.sum(r2_perm >= r2_obs) + 1) / (n_perm + 1)
        p_perm_mae = (np.sum(mae_perm <= mae_obs) + 1) / (n_perm + 1)
    
        perm_results[roi] = {
            "r2_obs": r2_obs,
            "mae_obs": mae_obs,
            "p_perm_r2": p_perm_r2,
            "p_perm_mae": p_perm_mae,
            "r2_perm": r2_perm,
            "mae_perm": mae_perm
        }
    
        metrics_sc[roi] = metrics_obs
        prediction_sc[roi] = pred_obs
        importances[roi] = imp_obs
     
        # save the results
        with open(results_file, "wb") as f:
            pickle.dump((metrics_sc, prediction_sc, importances, perm_results), f)
        print(f"Results saved to {results_file}")


In [ ]:
for roi, res in perm_results.items():
    print(f"\nROI: {roi}")
    print(f"Observed R²  = {res['r2_obs']:.3f}")
    print(f"Permuted p  = {res['p_perm_r2']:.4f}")
    print(f"Observed MAE = {res['mae_obs']:.2f}")
    print(f"Permuted p  = {res['p_perm_mae']:.4f}")

for roi in perm_results:
    plt.figure(figsize=(5,4))
    plt.hist(perm_results[roi]["r2_perm"], bins=30, color="lightgray")
    plt.axvline(perm_results[roi]["r2_obs"], color="red", linewidth=2,
                label=f"Observed R² = {perm_results[roi]['r2_obs']:.3f}")
    plt.xlabel("Permuted R²")
    plt.ylabel("Count")
    plt.title(f"Permutation test – {roi}")
    plt.legend()
    #plt.savefig(config["project_dir"] +  '/figures/f01_structural/revision_R1/spinalcord/spinal_age_prediction_perm_'+roi+'_r2.pdf', format='pdf')
    plt.show()


In [ ]:
for roi, res in perm_results.items():
    print(f"\nROI: {roi}")
    print(f"Observed R²  = {res['r2_obs']:.3f}")
    print(f"Permuted p  = {res['p_perm_r2']:.4f}")
    print(f"Observed MAE = {res['mae_obs']:.2f}")
    print(f"Permuted p  = {res['p_perm_mae']:.4f}")

for roi in perm_results:
    plt.figure(figsize=(5,4))
    plt.hist(perm_results[roi]["mae_perm"], bins=30, color="lightgray")
    plt.axvline(perm_results[roi]["mae_obs"], color="red", linewidth=2,
                label=f"Observed R² = {perm_results[roi]['mae_obs']:.3f}")
    plt.xlabel("Permuted mae")
    plt.ylabel("Count")
    plt.title(f"Permutation test – {roi}")
    plt.legend()
    plt.savefig(config["project_dir"] +  '/figures/f01_structural/revision_R1/spinalcord/spinal_age_prediction_perm_'+roi+'_mae.pdf', format='pdf')
    plt.show()

